In [1]:
import warnings
warnings.filterwarnings('ignore')

from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as ac
import astropy.units as au
from glob import glob
from spectral_cube import SpectralCube

from astropy.io import fits
import matplotlib as mpl
import pyregion
import aplpy
import math
import os
import pickle
from tqdm.auto import tqdm 

from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.nddata import Cutout2D


plt.rcParams["font.family"] = "sans-serif"
plt.rcParams["font.weight"] = "bold"
plt.rcParams["axes.labelweight"] = "bold"
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["ytick.direction"] = "in" 

In [2]:
"""Easy pickel"""

def save_pickle(a, filename): 
    with open(filename, 'wb') as handle:
        pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('[INFO] Saved to %s' %filename)
    return()

def load_pickle(filename):
    with open('%s'%filename, 'rb') as handle:
        b = pickle.load(handle)
    print('[INFO] Load %s' %filename)
    return(b)

In [3]:
hdus = load_pickle('../../analysis/catalouge/hdus_sample_v1.pickel')
hdus_scaled = load_pickle('../../analysis/catalouge/hdus_sample_v1.pickel')

[INFO] Load ../../analysis/catalouge/hdus_sample_v1.pickel
[INFO] Load ../../analysis/catalouge/hdus_sample_v1.pickel


In [17]:
def get_regions(regions_file, hdu):
    
    regions = pyregion.open(regions_file).as_imagecoord(hdu.header)
    n = len(regions)
    
    ra = np.empty(n)*np.nan*au.deg
    dec = np.empty(n)*np.nan*au.deg
    width = np.empty(n)*np.nan*au.deg
    height = np.empty(n)*np.nan*au.deg
    position = ['']*n
    
    for i, region in enumerate(regions):

        ra[i] = float(region.params[0].text)*au.deg
        dec[i] = float(region.params[1].text)*au.deg
        width[i] = region.params[2].degree*au.deg
        height[i] = region.params[3].degree*au.deg
        
    position = SkyCoord(ra=ra, dec=dec, frame='icrs')

    return({'ra':ra, 'dec':dec, 'width':width, 'height':height, 'position': position})

regions_file = './../../analysis/catalouge/sample_v1.reg'
regions = get_regions(regions_file, hdus['hst01_hdus'][0])

In [18]:
"""Converting to log for plotting"""

def get_scaling(data):
    data = (data-np.nanmin(data))/(np.nanmax(data)-np.nanmin(data))
    return(data)

# hdus_scaled = hdus.copy()
for key in tqdm(hdus_scaled.keys()): 
    n = len(hdus_scaled[key])
    for i in range(n):
        hdus_scaled[key][i].data = np.sqrt(get_scaling(hdus_scaled[key][i].data))

  0%|          | 0/24 [00:00<?, ?it/s]

In [14]:
for i in tqdm(range(n)):

    if i!=4: 
        continue
    
    ra = regions['ra'][i]
    dec = regions['dec'][i]
    width = regions['width'][i]
    height = regions['height'][i]
    radius = max([width,height])
    
    fig = plt.figure(figsize=(15, 12)) 
    fig.tight_layout()
    fig.subplots_adjust(wspace=0.02, hspace=0.02)
    
    ax = [''] * 28

    ax[0] = aplpy.FITSFigure(hdus['muscat_hdus'][i], subplot=(4,7,1), figure=fig)
    ax[1] = aplpy.FITSFigure(hdus['asscat1_hdus'][i], subplot=(4,7,2), figure=fig)
    ax[2] = aplpy.FITSFigure(hdus['asscat2_hdus'][i], subplot=(4,7,3), figure=fig)
    ax[3] = aplpy.FITSFigure(hdus['asscat3_hdus'][i], subplot=(4,7,4), figure=fig)
    
    ax[4] = aplpy.FITSFigure(hdus_scaled['musha_hdus'][i], subplot=(4,7,5), figure=fig)
    ax[5] = aplpy.FITSFigure(hdus_scaled['hst06_hdus'][i], subplot=(4,7,6), figure=fig)
    ax[6] = aplpy.FITSFigure(hdus_scaled['almaco_hdus'][i], subplot=(4,7,7), figure=fig)
    ax[7] = aplpy.FITSFigure(hdus_scaled['hst01_hdus'][i], subplot=(4,7,8), figure=fig)
    ax[8] = aplpy.FITSFigure(hdus_scaled['hst02_hdus'][i], subplot=(4,7,9), figure=fig)
    ax[9] = aplpy.FITSFigure(hdus_scaled['hst03_hdus'][i], subplot=(4,7,10), figure=fig)
    ax[10] = aplpy.FITSFigure(hdus_scaled['hst04_hdus'][i], subplot=(4,7,11), figure=fig)
    ax[11] = aplpy.FITSFigure(hdus_scaled['hst05_hdus'][i], subplot=(4,7,12), figure=fig)
    ax[12] = aplpy.FITSFigure(hdus_scaled['jwst01_hdus'][i], subplot=(4,7,13), figure=fig)
    ax[13] = aplpy.FITSFigure(hdus_scaled['jwst02_hdus'][i], subplot=(4,7,14), figure=fig)
    ax[14] = aplpy.FITSFigure(hdus_scaled['jwst03_hdus'][i], subplot=(4,7,15), figure=fig)
    ax[15] = aplpy.FITSFigure(hdus_scaled['jwst04_hdus'][i], subplot=(4,7,16), figure=fig)
    ax[16] = aplpy.FITSFigure(hdus_scaled['jwst05_hdus'][i], subplot=(4,7,17), figure=fig)
    ax[17] = aplpy.FITSFigure(hdus_scaled['jwst06_hdus'][i], subplot=(4,7,18), figure=fig)
    ax[18] = aplpy.FITSFigure(hdus_scaled['jwst07_hdus'][i], subplot=(4,7,19), figure=fig)
    ax[19] = aplpy.FITSFigure(hdus_scaled['jwst08_hdus'][i], subplot=(4,7,20), figure=fig)

    ax[12] = aplpy.FITSFigure(hdus_scaled['jwst09_hdus'][i], subplot=(4,7,21), figure=fig)
    ax[13] = aplpy.FITSFigure(hdus_scaled['jwst10_hdus'][i], subplot=(4,7,22), figure=fig)
    ax[14] = aplpy.FITSFigure(hdus_scaled['jwst11_hdus'][i], subplot=(4,7,23), figure=fig)
    ax[15] = aplpy.FITSFigure(hdus_scaled['jwst12_hdus'][i], subplot=(4,7,24), figure=fig)
    ax[16] = aplpy.FITSFigure(hdus_scaled['jwst13_hdus'][i], subplot=(4,7,25), figure=fig)
    ax[17] = aplpy.FITSFigure(hdus_scaled['jwst14_hdus'][i], subplot=(4,7,26), figure=fig)
    ax[18] = aplpy.FITSFigure(hdus_scaled['jwst15_hdus'][i], subplot=(4,7,27), figure=fig)
    ax[19] = aplpy.FITSFigure(hdus_scaled['jwst16_hdus'][i], subplot=(4,7,28), figure=fig)
    

    ax[0].show_colorscale(vmin=0, vmax=1e3, stretch='linear', cmap='turbo')
    ax[1].show_colorscale(vmin=0, vmax=1e3, stretch='linear', cmap='turbo')
    ax[2].show_colorscale(vmin=0, vmax=1e3, stretch='linear', cmap='turbo')
    ax[3].show_colorscale(vmin=0, vmax=1e3, stretch='linear', cmap='turbo')
    
    ax[4].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')    
    ax[5].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[6].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[7].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[8].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[9].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[10].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[11].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[12].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[13].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[14].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[15].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[16].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[17].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[18].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    ax[19].show_colorscale(pmin=1, pmax=99.99, stretch='linear', cmap='turbo')
    
    ax[0].add_label(0.5, 0.9, 'MUSE cat', relative=True)
    ax[1].add_label(0.5, 0.9, 'HST cat', relative=True)
    ax[2].add_label(0.5, 0.9, 'HST cat ', relative=True)    
    ax[3].add_label(0.5, 0.9, 'HST cat', relative=True)
    ax[4].add_label(0.5, 0.9, 'MUSE Ha', relative=True)
    ax[5].add_label(0.5, 0.9, 'HST Ha', relative=True)
    ax[6].add_label(0.5, 0.9, 'ALMA CO', relative=True)
    ax[7].add_label(0.5, 0.9, 'HST 275w', relative=True)
    ax[8].add_label(0.5, 0.9, 'HST 336w', relative=True)
    ax[9].add_label(0.5, 0.9, 'HST 435w', relative=True)
    ax[10].add_label(0.5, 0.9, 'HST 555w', relative=True)
    ax[11].add_label(0.5, 0.9, 'HST 814w', relative=True)
    ax[12].add_label(0.5, 0.9, 'JWST 200w', relative=True)
    ax[13].add_label(0.5, 0.9, 'JWST 300m', relative=True)
    ax[14].add_label(0.5, 0.9, 'JWST 335m', relative=True)
    ax[15].add_label(0.5, 0.9, 'JWST 360m', relative=True)
    ax[16].add_label(0.5, 0.9, 'JWST 770w', relative=True)
    ax[17].add_label(0.5, 0.9, 'JWST 1000w', relative=True)
    ax[18].add_label(0.5, 0.9, 'JWST 1130w', relative=True)
    ax[19].add_label(0.5, 0.9, 'JWST 2100w', relative=True)

    for j in range(len(ax)):
        
        ax[j].recenter(ra.value, dec.value, radius=radius.value/2.)
        label = '1arcsec'
        ax[j].add_scalebar(1/3600., label, color='black', corner='bottom right', linewidth=2)        
        ax[j].tick_labels.hide()
        ax[j].axis_labels.hide()
        ax[j].ticks.set_color('black')
        ax[j].set_nan_color('white')
        
        for k in range(20): 
            ax[j].show_circles(ra.value, dec.value, k/3600*0.2, edgecolor='black', facecolor='none', alpha=0.2, ls=':', linewidth=1)
        
    fig.tight_layout()
    fig.savefig('./../../analysis/catalouge_figs/cutout_%s.pdf' %i, bbox_inches='tight', dpi=300, transparent=True)  

  0%|          | 0/187 [00:00<?, ?it/s]

INFO: Auto-setting vmin to -4.873e-02 [aplpy.core]
INFO: Auto-setting vmax to  1.104e+00 [aplpy.core]
INFO: Auto-setting vmin to  5.183e-02 [aplpy.core]
INFO: Auto-setting vmax to  1.063e+00 [aplpy.core]
INFO: Auto-setting vmin to  2.862e-02 [aplpy.core]
INFO: Auto-setting vmax to  1.097e+00 [aplpy.core]
INFO: Auto-setting vmin to  1.007e-01 [aplpy.core]
INFO: Auto-setting vmax to  9.691e-01 [aplpy.core]
INFO: Auto-setting vmin to  8.234e-02 [aplpy.core]
INFO: Auto-setting vmax to  9.774e-01 [aplpy.core]
INFO: Auto-setting vmin to  5.200e-02 [aplpy.core]
INFO: Auto-setting vmax to  1.057e+00 [aplpy.core]
INFO: Auto-setting vmin to  1.026e-01 [aplpy.core]
INFO: Auto-setting vmax to  1.046e+00 [aplpy.core]
INFO: Auto-setting vmin to  5.798e-02 [aplpy.core]
INFO: Auto-setting vmax to  9.586e-01 [aplpy.core]
INFO: Auto-setting vmin to -1.202e-02 [aplpy.core]
INFO: Auto-setting vmax to  1.046e+00 [aplpy.core]
INFO: Auto-setting vmin to  4.741e-02 [aplpy.core]
INFO: Auto-setting vmax to  1.0

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7fd06b91c1f0> (for post_execute):


KeyboardInterrupt: 